In [17]:
import looker_sdk #Note that the pip install required a hyphen but the import is an underscore.

import os #We import os here in order to manage environment variables for the tutorial. You don't need to do this on a local system or anywhere you can more conveniently set environment variables.

import json #This is a handy library for doing JSON work.

In [18]:
os.environ["LOOKERSDK_BASE_URL"] = "" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #As of Looker v23.18+, the 3.0 and 3.1 versions of the API are removed. Use "4.0" here.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set.")

All environment variables set.


In [19]:
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

Looker SDK 4.0 initialized successfully.


In [20]:
response = sdk.all_lookml_models()

In [21]:
import pandas as pd

# Assuming you have a list of objects similar to the one described
# Let's say 'response' is your list containing these LookmlModel objects

# Define a function to extract the data including the nested 'explores'
def extract_data(response):
    # Prepare an empty list to store the flattened data
    flat_data = []
    
    # Iterate over each item in the original response
    for item in response:
        # Access the attributes directly or via the __dict__ if they are custom objects
        can_dict = item.can
        allowed_db_connection_names = item.allowed_db_connection_names
        explores = item.explores
        has_content = item.has_content
        label = item.label
        name = item.name
        project_name = item.project_name
        unlimited_db_connections = item.unlimited_db_connections
        
        # Now process each explore in explores
        for explore in explores:
            # Create a dictionary for each row of the final DataFrame
            row = {
                'can': can_dict,
                'allowed_db_connection_names': allowed_db_connection_names,
                'explore_name': explore.name,
                'explore_description': explore.description,
                'explore_label': explore.label,
                'explore_hidden': explore.hidden,
                'explore_group_label': explore.group_label,
                'has_content': has_content,
                'label': label,
                'model_name': name,
                'project_name': project_name,
                'unlimited_db_connections': unlimited_db_connections
            }
            flat_data.append(row)
    
    # Return the list of dictionaries which can be easily converted to a DataFrame
    return flat_data

# Suppose 'response' is your list of LookmlModel objects, convert it using the function
flat_data = extract_data(response)

# Create a DataFrame from the flattened data
df = pd.DataFrame(flat_data)

In [22]:
columns = ['model_name',
                'explore_name',
                'connection_name',
                'field_type',
                'view_name',
                'field_name',
                'type',
                'description',
                'sql']

df_final = pd.DataFrame(columns = columns)

In [23]:
for index, row in df.iterrows():
    model_name = row['model_name']
    explore_name = row['explore_name']
    for row in get_field_values(model_name, explore_name):
        df_final.loc[len(df_final)] = row

In [24]:
df_final

,model_name,explore_name,connection_name,field_type,view_name,field_name,type,description,sql
0,basic_ecomm,basic_order_items,sample_bigquery_connection,Dimension,Basic Order Items,Created At Date,date_date,,${TABLE}.created_at
1,basic_ecomm,basic_order_items,sample_bigquery_connection,Dimension,Basic Order Items,Created At Month,date_month,,${TABLE}.created_at
2,basic_ecomm,basic_order_items,sample_bigquery_connection,Dimension,Basic Order Items,Created At Quarter,date_quarter,,${TABLE}.created_at
3,basic_ecomm,basic_order_items,sample_bigquery_connection,Dimension,Basic Order Items,Created At Raw,date_raw,,${TABLE}.created_at
4,basic_ecomm,basic_order_items,sample_bigquery_connection,Dimension,Basic Order Items,Created At Time,date_time,,${TABLE}.created_at
...,...,...,...,...,...,...,...,...,...
576,eder_demo,procedure_cost,looker-genai,Dimension,Procedure Cost,Procedure_name,string,,${TABLE}.procedure
577,eder_demo,procedure_cost,looker-genai,Dimension,Procedure Cost,Procedure_price,number,,${TABLE}.price
578,eder_demo,procedure_cost,looker-genai,Measure,Anomaly,Count,count,,None
579,eder_demo,procedure_cost,looker-genai,Measure,Appointments,Count,count,,None
